### 작업 순서
1. STL10 데이터셋 클래스 정의
- 이전에 CustomImageDataset 클래스를 구현한 것처럼, STL10 데이터를 위한 데이터셋 클래스 생성
- 데이터셋 클래스는 STL10의 이미지와 레이블을 불러오고, 필요한 전처리를 수행
2. MLP(Multi-Layer Perceptron) 모델 구성
- STL10 데이터셋을 학습하기 위해 다층 퍼셉트론(MLP) 모델로 구현
- 입력층 → 은닉층(여러 층) → 출력층으로 구성
3. 모델 학습
- 데이터셋을 사용해 MLP 모델을 학습
- 손실 함수와 옵티마이저를 설정, 에폭별 학습을 수행
4. 테스트 및 Accuracy 측정
- 테스트 데이터셋으로 모델의 성능(Accuracy)을 평가

## 1. STL10 데이터셋 클래스 정의
STL10 데이터셋을 학습 및 테스트 데이터로 로드하고, 파이토치 데이터로더를 통해 배치 단위로 데이터를 준비하는 작업을 수행   
해당 데이터셋을 모댈 학습에 사용할 수 있도록 전처리와 로더 생성까지 포함

In [18]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 데이터 변환 정의
# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((96, 96)),  # STL10 크기에 맞
    transforms.ToTensor(),       # 이미지를 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화
])

# STL10 학습 및 테스트 데이터셋 다운로드
train_dataset = datasets.STL10(root="data", split="train", download=True, transform=transform)
test_dataset = datasets.STL10(root="data", split="test", download=True, transform=transform)
val_dataset = datasets.STL10(root='./data', split='test', transform=transform, download=True)

# DataLoader 생성 (64개의 배치로 나누고, 학습 데이터를 셔플)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


#### 데이터 변환 정의
transforms.Compose:  
- 여러 변환을 순서대로 적용하기 위해 사용  
transforms.ToTensor():  
- 이미지를 파이토치 텐서로 변환    
- 픽셀 값을 [0,255] 에서 [0,1] 범위로 정규화
#### STL10 데이터셋 로드
STL10 데이터셋:   
- 이미지 크기: 96x96 RGB(3채널)   
- 클래스: 10개 클래스   
- 데이터 구성:  
    - 학습 데이터: 5000개
    - 테스트 데이터: 8000개
매개변수:
- root="data":
    - 데이터셋이 저장될 디렉토리 경로
    - 디렉토리에 데이터가 없으면,download= True 로 다운
- split="train"/split="test":
    - split 매개변수를 사용해 학습 데이터와 테스트 데이터를 나눔
- transform=transform:
    - 데이터에 정의한 변환(ToTensor)을 적용

#### DataLoader 생성
DataLoader:
- PyTorch에서 제공하는 데이터로더로 데이터를 배치 단위로 나눠 모델 학습에 적합한 형태로 제공
매개변수:
- train_dataset/ test_dataset:
    - STL10의 학습 및 테스트 데이터셋 객체
- batch_size = 64:
    - 한 번에 반환할 데이터 샘플의 개수(64개)
    - 학습 데이터 5,000개 → 5,000 ÷ 64 ≈ 78개의 배치
- shuffle = True:
    - 학습 데이터는 에폭마다 랜덤하게 섞어 학습의 다양성 보장
    - 모델이 데이터 순서에 의존하지 않도록 방지
- shuffle = False:  
    - 테스트 데이터는 순서를 유지

# 어떤 관점으로 푸려고 했느냐
## hyperparameter를 구하기 위해 한 것들

> 1. 학습과 모델의 성능에 영향을 주는 Activation Function / learning rate / Loss function 값들을 각각 지정해봄
2. 학습시키고, evaluation을 하면서 Loss를 찍어봄.
3. loss가 낮아지는 추세가 계속 될 때까지 학습
3. loss가 낮아지다가 다시 올라가는 구간에서 다른 parameter설정과의 비교 및 원인 분석
4. 다른 parameter설정과의 비교

In [19]:
from torch import nn

# MLP 모델 정의
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, n_layers, output_size, activation_function):
        """
        Flexible한 MLP Model
        Args:
            input_size: Input 차원
            hidden_sizes : List of hidden layer sizes.
            output_size: Output 차원
            activation_fn : 여러개 중 선택해보기
        """

        super(MLP, self).__init__()


        layers = []
        previous_size = input_size

        # Hidden layers 생성
        for _ in range(n_layers):
            layers.append(nn.Linear(previous_size, hidden_size))  # Linear layer
            layers.append(activation_function())  # Activation function
            previous_size = hidden_size

        # Output layer 생성
        layers.append(nn.Linear(previous_size, output_size))

        # layers라는 list에 넣었던 입력층의 결과들을 다시 Sequential 모델로 결합
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# STL10 이미지 크기: 96x96, 채널 3 (3채널 이미지를 펼침)
input_size = 96 * 96 * 3
hidden_size = 128 # hidden unit의 개수를 128에서 점점 줄여서 나가도 됨 --> 특징 추출을 세밀화
n_layers = 3  # Hidden layer 개수
output_size = 10  # 클래스 개수


activation_function = nn.ReLU # 활성화 함수 설정

model = MLP(input_size, hidden_size, n_layers, output_size, activation_function)

In [20]:
from torch import optim

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# Optimizer 설계
lr = 0.001

# 에포크 설정
epochs = 100
optimizer = optim.Adam(model.parameters(), lr)


# epoch 별 train loss / validation loss / accuracy loss 저장할 list --> 매번 출력하여 추이 파악
list_epoch = []
list_train_loss = []
list_val_loss = []
list_acc = []
list_acc_epoch = []




# Train
# 학습 함수 정의
def train_model(model, dataloader, val_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in dataloader:

            images = images.view(images.size(0), -1)  # 이미지를 1D 벡터로 변환
            optimizer.zero_grad()
            outputs = model(images)  # 모델에 입력

            loss = criterion(outputs, labels)  # 손실 계산
            loss.backward()  # 역전파
            optimizer.step()  # 파라미터 업데이트
            train_loss = train_loss + loss.item()

        train_loss = train_loss / len(dataloader)
        list_train_loss.append(train_loss)
        list_epoch.append(epoch)

        # validation loss
        val_loss = evaluate_loss(model, val_loader, criterion)
        list_val_loss.append(val_loss)

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


# loss function 평가
def evaluate_loss(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.view(images.size(0), -1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)



# Evaluation
# 평가 함수 정의
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # 평가 시에는 그래디언트 계산 비활성화
        for images, labels in dataloader:
            images = images.view(images.size(0), -1)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # 예측 값 추출
            correct = correct + (predicted == labels).sum().item()
            total = total + labels.size(0)
    accuracy = correct / total
    list_acc.append(accuracy)
    list_acc_epoch.append(len(list_epoch))
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return accuracy

# print('Epoch: {}, Train Loss: {}, Val Loss: {}, Test Acc: {}%'.format(i, train_loss, val_loss, acc*100))

In [21]:
# 모델 학습
train_model(model, train_loader, val_loader, criterion, optimizer, epochs)

# 모델 평가
accuracy = evaluate_model(model, test_loader)

Epoch [1/100], Train Loss: 1.8931, Val Loss: 1.8248
Epoch [2/100], Train Loss: 1.6378, Val Loss: 1.7176
Epoch [3/100], Train Loss: 1.4348, Val Loss: 1.7531
Epoch [4/100], Train Loss: 1.2661, Val Loss: 1.7350
Epoch [5/100], Train Loss: 1.1256, Val Loss: 1.8816
Epoch [6/100], Train Loss: 0.9557, Val Loss: 1.9219
Epoch [7/100], Train Loss: 0.7563, Val Loss: 2.1898
Epoch [8/100], Train Loss: 0.6202, Val Loss: 2.3719
Epoch [9/100], Train Loss: 0.5576, Val Loss: 2.5618
Epoch [10/100], Train Loss: 0.4750, Val Loss: 2.7176
Epoch [11/100], Train Loss: 0.3792, Val Loss: 3.0919
Epoch [12/100], Train Loss: 0.4975, Val Loss: 2.8794
Epoch [13/100], Train Loss: 0.2524, Val Loss: 3.1523
Epoch [14/100], Train Loss: 0.2660, Val Loss: 3.4166
Epoch [15/100], Train Loss: 0.1761, Val Loss: 3.7746
Epoch [16/100], Train Loss: 0.1919, Val Loss: 3.6923
Epoch [17/100], Train Loss: 0.1933, Val Loss: 3.7389
Epoch [18/100], Train Loss: 0.1670, Val Loss: 3.9706
Epoch [19/100], Train Loss: 0.1678, Val Loss: 3.9592
Ep

# 느낀점
Train Loss가 점점 줄어들었지만, val loss는 늘어났음.
STL에서 validation 데이터를 제공하지 않아서, 이를 확인하기 위해서 각 epoch마다 train loss, validation loss를 찍어보았음.
epoch가 반복함에 따라 모델이 학습되지 않은 데이터에서는 오버피팅이 일어나고 있다는 것을 알 수 있었으나, accuracy는 올라가는 발견할 수 있었음.

이 코드에서는 아무래도 validation 과정이 없어서 모델이 학습과정에서 잘 하고 있는지를 잘 파악하지 못 하고 있더라고요. (epoch 마다 validation loss를 출력했더니 학습마다 loss가 증가함)

그래서 제가 추가적으로 해볼 거는 train 데이터 중 일부를 validation데이터로 분리(데이터 비중은 최소 0.1~ 최대 0.2)하고 각각의 학습마다 validation 거쳐서 오버피팅 막아보고자 했다.(비중에 따른 cross validation을 적용하여 각각의 값을 비교하고 우수한 것을 고르는 비교하여 최적의 k값을 고를 수 있겠는데 이 과정은 한다해도 다음 step에서 진행하는 게 맞는 것 같음)

우선 위 과정만 해도 val_loss에서 오버피팅은 줄어들 것 같았음
